#### Create score distribution exhibits for CIS189.



In [ ]:

from random import sample
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Configuration ----------------------------------------------------------------

csv_path = "G:/My Drive/CIS189/Spring-2024-CIS189-Grades.csv"
module = 1

# ------------------------------------------------------------------------------

dfall = pd.read_csv(csv_path)

df = dfall[dfall.module==module].reset_index(drop=True)

print(df.shape)



In [ ]:


dfassign = df[["module", "assignment", "desc"]].drop_duplicates()

# Get number of NAs for current module assignments. 
dfna = (
    df
    .groupby(["module", "assignment"])
    .count()
    .rsub(df.groupby("assignment").size(), axis=0)["grade"]
    .reset_index(drop=False)
    .rename({"grade": "missing"}, axis=1)
    )

# Merge dfassin with dfna. 
dfassign = dfassign.merge(dfna, on=["module", "assignment"], how="left")

dfassign.head()


In [ ]:
df[(df.assignment==assign) & (df.grade>0)]["grade"].min().item()

In [ ]:

# color = "#E02C70"
cmap = "winter"


n_facets = dfassign.shape[0]
fcolors = mpl.colormaps.get_cmap(cmap)
colors_rgba = [fcolors(ii) for ii in np.linspace(0, 1, n_facets)]
colors_hex = [mpl.colors.to_hex(ii, keep_alpha=False) for ii in colors_rgba]

fig, ax = plt.subplots(1, n_facets, figsize=(11, 5.0), sharey=True, sharex=True, tight_layout=True) 

for ii, color in enumerate(colors_hex):
    module = dfassign.at[ii, "module"]
    assign = dfassign.at[ii, "assignment"]
    desc = dfassign.at[ii, "desc"]
    nbr_students = df[(df.module==module) & (df.assignment==assign)].shape[0]
    possible = df[(df.module==module) & (df.assignment==assign)]["possible"].values[0]
    dfgrades = pd.DataFrame(np.arange(0, possible + 1, dtype=int), columns=["grade"])
    gg = df[df.assignment==assign].fillna(0).groupby("grade", as_index=False).size()
    gg = dfgrades.merge(gg, on="grade", how="left").fillna(0).rename({"size": "n"}, axis=1)
    zs = int(gg[gg.grade==0]["n"].item())
    dfavg = gg[gg.grade!=0]
    avg_score = (dfavg.grade * dfavg.n).sum() / dfavg.n.sum()
    # min_score = df[df.assignment==assign].fillna(0)["grade"].min()
    min_score = df[(df.assignment==assign) & (df.grade>0)]["grade"].min().item()
    max_score = df[df.assignment==assign].dropna(how="any")["grade"].max()
    gg = gg.assign(grade=gg["grade"].astype(str), n=gg["n"].astype(int))
    gg.plot.bar(ax=ax[ii], color=color)

    title_str = f"{desc}"

    ax[ii].set_title(title_str, fontsize=11, weight="normal")

    ax[ii].set_xticklabels(gg["grade"].values, rotation=0)
    ax[ii].set_xlabel("grade", fontsize=9)
    ax[ii].set_ylabel("nbr. students", fontsize=9)
    ax[ii].tick_params(axis="x", which="major", direction='in', labelsize=9)
    ax[ii].tick_params(axis="y", which="major", direction='in', labelsize=9)
    ax[ii].xaxis.set_ticks_position("none")
    ax[ii].yaxis.set_ticks_position("none")

    # ax[ii].grid(True)   
    # ax[ii].set_axisbelow(True) 
    missing = int(gg[gg.grade=="0"]["n"].item())
    prop_missing = missing / nbr_students
    avg_desc = f"- avg. score : {avg_score:.2f}"
    min_desc = f"- min. score : {min_score:.1f}"
    max_desc = f"- max. score : {max_score:.1f}"
    na_desc  = f"- non-submits: {missing} ({prop_missing:.2%})"

    ax[ii].annotate(
        avg_desc, xy=(.05, .925), xycoords="axes fraction", ha="left", va="bottom", 
        fontsize=10, rotation=0, weight="bold", color="#000000"
        ) 
    ax[ii].annotate(
        min_desc, xy=(.05, .875), xycoords="axes fraction", ha="left", va="bottom", 
        fontsize=10, rotation=0, weight="bold", color="#000000"
        ) 
    ax[ii].annotate(
        max_desc, xy=(.05, .825), xycoords="axes fraction", ha="left", va="bottom", 
        fontsize=10, rotation=0, weight="bold", color="#000000"
        ) 
    ax[ii].annotate(
        na_desc, xy=(.05, .775), xycoords="axes fraction", ha="left", va="bottom", 
        fontsize=10, rotation=0, weight="bold", color="#FF0000"
        ) 
    ax[ii].get_legend().remove()


plt.suptitle(f"CIS189 Module {module} Assignments", weight="bold", size=12)
plt.show()
